# Channels

> Django Channels extends Django to handle WebSockets, HTTP2, and other asynchronous protocols. It allows you to add real-time functionalities like chat, notifications, live updates, and more, while still using Django's familiar structure.


- skip_showdoc: true
- skip_exec: true

## **1. What is Django Channels?**
Django Channels allows Django to support:
- **WebSockets:** For real-time communication between server and client.
- **Long-lived connections:** Such as HTTP2 or custom protocols.
- **Background tasks:** Offload tasks to background workers.

It integrates Django's standard request/response cycle with asynchronous communication protocols, enabling you to build real-time applications.

---

## **2. Why Use Django Channels?**
- **Real-Time Features:** Chat applications, live notifications, collaborative editing, etc.
- **Asynchronous Tasks:** Run background tasks without blocking the main request/response cycle.
- **WebSocket Support:** Full-duplex communication channel.

---

## **3. How Django Channels Works**
- **ASGI (Asynchronous Server Gateway Interface):** Django Channels uses ASGI, which is the asynchronous counterpart to WSGI.
- **Channels and Consumers:**
  - **Channels:** Queues for messages.
  - **Consumers:** Handlers for messages that run asynchronously.
- **Routing:** Similar to Django’s URL routing but for WebSocket connections.
- **Layer:** Channels uses a "layer" (e.g., Redis) for cross-process communication.

---

## **4. Key Components**
1. **ASGI Application:** Entry point for asynchronous communication.
2. **Consumers:** Async functions or classes that handle WebSocket connections.
3. **Routing:** Maps WebSocket paths to Consumers.
4. **Channel Layers:** Manages message communication between consumers (commonly using Redis).
5. **Middleware:** Similar to Django's middleware but for ASGI.

---

## **5. Installation and Setup**
**Requirements:**
- Django 3.0+
- Channels 4.0+ (latest)
- ASGI server (e.g., Daphne, Uvicorn)

### **1. Install Channels and Redis:**
```bash
pip install channels channels-redis
```

### **2. Update Django Settings:**
```python
# settings.py
INSTALLED_APPS = [
    ...,
    'channels',
]

# Point to the ASGI application
ASGI_APPLICATION = 'myproject.asgi.application'

# Channel Layer Configuration
CHANNEL_LAYERS = {
    'default': {
        'BACKEND': 'channels_redis.core.RedisChannelLayer',
        'CONFIG': {
            'hosts': [('127.0.0.1', 6379)],  # Redis server location
        },
    },
}
```

### **3. Create ASGI Configuration:**
```python
# asgi.py
import os
from django.core.asgi import get_asgi_application
from channels.routing import ProtocolTypeRouter, URLRouter
from channels.auth import AuthMiddlewareStack
from myapp.routing import websocket_urlpatterns

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'myproject.settings')

application = ProtocolTypeRouter({
    "http": get_asgi_application(),
    "websocket": AuthMiddlewareStack(
        URLRouter(
            websocket_urlpatterns
        )
    ),
})
```

### **4. Create Routing:**
```python
# myapp/routing.py
from django.urls import path
from .consumers import ChatConsumer

websocket_urlpatterns = [
    path('ws/chat/<str:room_name>/', ChatConsumer.as_asgi()),
]
```

### **5. Create Consumer:**
```python
# myapp/consumers.py
import json
from channels.generic.websocket import AsyncWebsocketConsumer

class ChatConsumer(AsyncWebsocketConsumer):
    async def connect(self):
        self.room_name = self.scope['url_route']['kwargs']['room_name']
        self.room_group_name = f"chat_{self.room_name}"

        # Join room group
        await self.channel_layer.group_add(
            self.room_group_name,
            self.channel_name
        )
        await self.accept()

    async def disconnect(self, close_code):
        # Leave room group
        await self.channel_layer.group_discard(
            self.room_group_name,
            self.channel_name
        )

    async def receive(self, text_data):
        text_data_json = json.loads(text_data)
        message = text_data_json['message']

        # Send message to room group
        await self.channel_layer.group_send(
            self.room_group_name,
            {
                'type': 'chat_message',
                'message': message
            }
        )

    async def chat_message(self, event):
        message = event['message']

        # Send message to WebSocket
        await self.send(text_data=json.dumps({
            'message': message
        }))
```

### **6. Frontend WebSocket Connection (JavaScript):**
```js
const roomName = JSON.parse(document.getElementById('room-name').textContent);
const chatSocket = new WebSocket(
    'ws://' + window.location.host + '/ws/chat/' + roomName + '/'
);

chatSocket.onmessage = function(e) {
    const data = JSON.parse(e.data);
    document.querySelector('#chat-log').value += (data.message + '\n');
};

chatSocket.onclose = function(e) {
    console.error('Chat socket closed unexpectedly');
};

document.querySelector('#chat-message-input').focus();
document.querySelector('#chat-message-input').onkeyup = function(e) {
    if (e.keyCode === 13) {  // Enter key
        const messageInputDom = document.querySelector('#chat-message-input');
        const message = messageInputDom.value;
        chatSocket.send(JSON.stringify({
            'message': message
        }));
        messageInputDom.value = '';
    }
};
```

### **7. Run the ASGI Server:**
```bash
daphne -b 0.0.0.0 -p 8000 myproject.asgi:application
```

---

## **6. Key Concepts in Channels**
### 1. **Consumers:**
- **WebSocketConsumer:** Synchronous consumer for WebSocket connections.
- **AsyncWebsocketConsumer:** Asynchronous version for non-blocking connections.
- **JsonWebsocketConsumer:** Extends AsyncWebsocketConsumer to work with JSON messages.

### 2. **Routing:**
- Similar to Django's URL routing but for WebSocket connections.
- Supports path converters like `<str:room_name>`.

### 3. **Channel Layers:**
- **Backends:** Redis, In-memory (for development/testing).
- **Use Case:** Pub/Sub messaging between consumers.
- **Example:**
    ```python
    await self.channel_layer.group_send(
        'group_name',
        {
            'type': 'chat_message',
            'message': message
        }
    )
    ```

### 4. **Authentication:**
- `AuthMiddlewareStack`: Adds user authentication to WebSocket connections.
- Works with Django's authentication system.

---

## **7. Deployment Considerations**
- **ASGI Servers:** Daphne or Uvicorn.
- **Redis Server:** For production-ready Channel Layers.
- **Scaling:**
  - Use multiple Daphne/Uvicorn instances behind a load balancer.
  - Horizontal scaling with Redis as the message broker.

**Example using Daphne with Daphne and Redis:**
```bash
daphne -b 0.0.0.0 -p 8000 myproject.asgi:application
```

---

## **8. Debugging Tips:**
- **Common Issues:**
  - Connection closed before receiving handshake response.
  - Error 403: CSRF token missing or incorrect.
- **Solutions:**
  - Check CORS and CSRF settings in Django.
  - Ensure Redis server is running and accessible.
  - Use `channels.layers.get_channel_layer()` to debug channel layers.

---

## **9. Real-World Use Cases**
- **Chat Applications:** Real-time messaging between users.
- **Live Notifications:** Push notifications for events.
- **Collaborative Editing:** Real-time document editing.
- **Online Games:** Real-time multiplayer games.
- **Live Data Feeds:** Stock prices, sports scores, etc.

---